In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd

In [2]:
class Ball:
    
    def __init__(self):

        self.x = -1
        self.y = np.random.uniform(-1,0)
        self.vx = np.random.uniform(0.1,0.4)
        self.vy =  np.random.uniform(1.2,1.5)

        self.g = 0.9
    
    def next_step(self,dt=1):
        x0, y0 = self.x, self.y
        self.vy = self.vy-self.g*dt
        
        dx = self.vx*dt
        dy = self.vy*dt
        self.x = x0+dx
        self.y = y0+dy
        if self.y < -1:
            self.y = -1
            self.vy = 0.9*abs(self.vy)
    def get_pos(self):
        return self.x, self.y
    def measure_pos(self):
        x_rand = self.x + 0.02*np.random.normal()
        y_rand = self.y + 0.02*np.random.normal()
        return x_rand,y_rand
    
    def get_trajectory(self):
        traj = []
        measured = []

        for ti in range(80):
            traj.append(self.get_pos())
            measured.append(self.measure_pos())
            self.next_step(0.2)
            
        traj = np.array(traj)
        measured = np.array(measured)
        self.__init__()
        return traj, measured

In [3]:
df = pd.DataFrame(columns=['true trajectory','measured trajectory','seq_length']) 


ball = Ball()
for i in range(10000):
    
    traj, measured = ball.get_trajectory()
    
    measured = measured[traj[:,0] <= 1]
    traj = traj[traj[:,0] <= 1]

    df.loc[i] = [0,0,0]
    df.loc[i]['true trajectory'] = traj
    df.loc[i]['measured trajectory'] = measured
    df.loc[i]['seq_length'] = len(traj)

In [14]:
test_df = pd.DataFrame(columns=['true trajectory','measured trajectory','seq_length']) 


ball = Ball()
for i in range(1000):
    
    traj, measured = ball.get_trajectory()
    
    measured = measured[traj[:,0] <= 1]
    traj = traj[traj[:,0] <= 1]

    test_df.loc[i] = [0,0,0]
    test_df.loc[i]['true trajectory'] = traj
    test_df.loc[i]['measured trajectory'] = measured
    test_df.loc[i]['seq_length'] = len(traj)

In [5]:
df.to_hdf('bouncing_ball.h5',key='df')

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['true trajectory', 'measured trajectory', 'seq_length']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [15]:
test_df.to_hdf('bouncing_ball_test.h5',key='df')

In [6]:
%%capture
from matplotlib import animation,rc
from IPython.display import HTML


traj, measured,_ = df.loc[0]

fig, ax = plt.subplots(figsize=(8,8))
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)

line, = ax.plot([],[],lw=2)
scat = ax.scatter([],[])

def init():
    line.set_data([],[])
    scat.set_offsets(np.zeros((1,2)))
    return (line,scat,)

def animate(i):
    x = traj[:,0][:i]
    y = traj[:,1][:i]
    line.set_data(x,y)
    scat.set_offsets(measured[:i])
    return ( line,)

anim = animation.FuncAnimation(fig,animate,init_func=init,frames=80,interval=30,blit=True)

In [7]:
HTML(anim.to_jshtml())

In [8]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)


anim.save('im.mp4', writer=writer)

In [12]:
def pick_random_chunk(sample):
    
    outputlength = 10
    
    traj = sample['true trajectory']
    measured = sample['measured trajectory']
    
    seq_len = sample['seq_length']
    
    sub_sample_len = np.random.randint(5,10)
    
    start_idx = np.random.randint(0,min(seq_len-outputlength-sub_sample_len,10))
    
    #print('len ',seq_len,'start ',start_idx,' sub_sample_len ',sub_sample_len)
    input_idx = range(start_idx,start_idx+sub_sample_len)
    
    target_idx = range(start_idx+sub_sample_len,start_idx+sub_sample_len+outputlength)
    
    input_arr = measured[input_idx]
    target_arr = traj[target_idx]
    
    return input_arr,target_arr,sub_sample_len

In [13]:
chunk_df = pd.DataFrame(columns=['input_arr','target_arr','seq_length']) 

for i in range(10000):
    sample = df.loc[i]
    
    input_arr,target_arr,sub_sample_len = pick_random_chunk(sample)

    chunk_df.loc[i] = [0,0,0]
    chunk_df.loc[i]['input_arr'] = input_arr
    chunk_df.loc[i]['target_arr'] = target_arr
    chunk_df.loc[i]['seq_length'] = sub_sample_len

In [17]:
chunk_df.to_hdf('bouncing_ball_chunks.h5',key='df')

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['input_arr', 'target_arr', 'seq_length']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [18]:
chunk_test_df = pd.DataFrame(columns=['input_arr','target_arr','seq_length']) 

for i in range(len(test_df)):
    sample = test_df.loc[i]
    
    input_arr,target_arr,sub_sample_len = pick_random_chunk(sample)

    chunk_test_df.loc[i] = [0,0,0]
    chunk_test_df.loc[i]['input_arr'] = input_arr
    chunk_test_df.loc[i]['target_arr'] = target_arr
    chunk_test_df.loc[i]['seq_length'] = sub_sample_len

In [19]:
chunk_test_df.to_hdf('bouncing_ball_chunks_test.h5',key='df')

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['input_arr', 'target_arr', 'seq_length']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
